# Adaptive partitioning algorithm for mutual information estimation

Python implementation of mutual information estimation, where adaptive partitioning strategy is applied.

## Reference
- Darbellay, G. A., & Vajda, I. (1999). Estimation of the information by an adaptive partitioning of the observation space. IEEE Transactions on Information Theory, 45(4), 1315-1321.

In [1]:
import numpy as np
from minfo.mi_float import mutual_info as mi_cy
from minfo.mi_float import TDMI as TDMI_cy
from mutual_info import mutual_info as mi_py

def TDMI_py(dat, n):
    """Time-delay mutual information estimator. (Pure Python Version)
    
    Parameters:
        dat (np.ndarray) : 2D array of time series with 2 column.
            each column is a variable, and each row is a sample.
        n (int) : number of delays, including zero time-lag case.

    Returns:
        np.ndarray : 1d array of delayed mutual information series.

    """
    tdmi = np.zeros(n)
    N = dat.shape[0]
    for i in range(n):
        dat_buffer = np.zeros((N-i, 2))
        dat_buffer[:,0] = dat[:N-i,0]
        dat_buffer[:,1] = dat[i:,1]
        tdmi[i] = mi_py(dat_buffer)
    return tdmi

In [2]:
# generate data
np.random.seed(2022)
n = 24000
dat = np.random.rand(n,2)
dat[:,1] += dat[:,0]

## Test mutual information estimation with different algorithm and implementations

In [3]:
print('[INFO]: Testing mi_adaptive (Python) ...')
%timeit mi_py(dat)

[INFO]: Testing mi_adaptive (Python) ...
28 ms ± 4.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
# run first to excute JIT compling
mi_cy(dat[:,0], dat[:,1], bins=50,)

print('[INFO]: Testing mi_uniform (Numba) ...')
%timeit mi_cy(dat[:,0], dat[:,1], bins=50,)

[INFO]: Testing mi_uniform (Numba) ...
226 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
print('[INFO]: Testing mi_adaptive (Cython) ...')
%timeit mi_cy(dat[:,0], dat[:,1], algorithm='adaptive')


[INFO]: Testing mi_adaptive (Cython) ...
9.32 ms ± 237 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Test time-delayed mutual information estimation with different algorithm and implementations

In [6]:
n_delay = 100

In [7]:
print('[INFO]: Testing tdmi_adaptive (Python) ...')
%timeit TDMI_py(dat, n_delay)


[INFO]: Testing tdmi_adaptive (Python) ...
593 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# run first to excute JIT compling
print('[INFO]: Testing tdmi_uniform (Numba) ...')
TDMI_cy(dat[:,0], dat[:,1], n_delay, bins=50)
%timeit TDMI_cy(dat[:,0], dat[:,1], n_delay, bins=50)


[INFO]: Testing tdmi_uniform (Numba) ...
5.77 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
print('[INFO]: Testing tdmi_adaptive (Cython) ...')
%timeit TDMI_cy(dat[:,0], dat[:,1], n_delay, algorithm='adaptive')

[INFO]: Testing tdmi_adaptive (Cython) ...
93.4 ms ± 8.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
